In [1]:
import numpy as np

samples = np.loadtxt("samples.csv", delimiter=",")
print(samples.shape)
# 1000 samples with 100 features each and a label

(1000, 101)


In [2]:
import cvxpy as cp

x = samples[:, :-1]
y = samples[:, -1]
#print(x[1], y[1])
#print(np.exp(x))
#raise error
# Create two scalar optimization variables.
w = cp.Variable(x.shape[1])
# no constraints
# (1/x.shape[0])*
# Form objective.
log = cp.atoms.elementwise.logistic.logistic(cp.multiply(-y, x@w))
obj = cp.Minimize((1/x.shape[0])*cp.sum(log))
#obj = cp.Minimize((1/x.shape[0])*np.sum([np.log(1+np.exp(-y[i]*np.dot(w, x[i]))) for i in range(len(y))]))

# Form and solve problem.
prob = cp.Problem(obj)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", w.value)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


status: optimal_inaccurate
optimal value 2.906898597653035e-06
optimal var [ 5.06588574e-02 -5.95797114e-02  1.82092115e+00 -2.44486554e-01
 -2.36757664e+00  8.38680696e-02  6.47585633e-01 -1.16286388e+00
 -3.70768858e-01 -2.83654378e-01 -9.72149296e-01 -2.17355877e+00
 -1.45820006e+00  5.37591065e-01 -1.14187019e-02  1.68736075e+00
 -5.84488898e-01 -2.01471451e-02 -9.00750146e-01  6.54764665e-02
 -1.96524557e+00  2.07060530e+00 -1.81903029e+00 -9.00161894e-01
 -2.43145262e+00  2.11478967e+00  1.11655441e-01 -9.57377106e-01
  1.76654277e+00 -2.63104979e+00  1.16186782e-02  6.88893336e-02
  1.51212849e-01  1.30492858e+00 -5.18994707e-01 -1.62048276e+00
 -2.61645815e-01 -2.53919348e+00  3.47105572e-01 -1.95835638e+00
  1.18701740e+00  6.86506249e-01  4.10215960e-01 -1.58815664e+00
 -1.75610835e+00  2.10212552e+00 -2.01297025e+00 -2.50256717e+00
 -1.94086419e+00  1.15247817e+00 -2.76731487e-01 -2.94485828e-01
  4.98048361e-02 -1.45990055e+00 -1.44015739e+00 -1.00856561e+00
 -1.38490363e+0

/Users/tomasmatteson/miniconda3/envs/jup-env/lib/python3.9/site-packages/cvxpy/problems/problem.py:1296: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [3]:
import matplotlib.pyplot as plt



def momentum_update(learning_rate, gamma, old_g, w, Y, X):
    full_gradient = []
    for i in range(X.shape[0]):
        gradient = (1/(1+np.exp((Y[i] * (X[i] @ w))))) * (X[i] * -Y[i]) 
        full_gradient.append(gradient)
    gradient = np.sum(full_gradient, axis=0) /X.shape[0]
    gradient = np.reshape(gradient, (len(gradient), 1))
    g_t = (1- gamma)*old_g + gamma*gradient
    #w = w - learning_rate*g_t(gamma, old_g, w, Y, X)
    w = w- learning_rate*g_t
    assert w.shape == (100, 1), "w's shape has been changed"
    #return w, g_t(gamma, old_g, w, Y, X)
    return w, g_t
                  
def objective(Y, X, w):
    # another try-except here?
    np.seterr(over='ignore')
    obj = (1/1000)*np.sum(np.log(1+np.exp(np.multiply(-Y, X@w))))
    return obj

def calc_diff(w, X, Y):
    return np.log(objective(Y, X, w) - 2.906898597653035e-06)

def mgd(gamma, learning_rate, X, Y):    
    w = np.zeros((X.shape[1], 1))
    old_g = np.zeros((X.shape[1], 1))
    diffs = []
    for i in range(1000):
#         x, y = X[i], Y[i]
#         x = np.reshape(x, (1, len(x)))
#         y = np.reshape(y, (1, len(y)))
        w, old_g = momentum_update(learning_rate, gamma, old_g, w, Y, X)
#         if i % 100 == 0:
#             print(max(abs(X@w)))
        diffs.append(calc_diff(w, X, Y))
    return (diffs, objective(Y, X, w))

def collect_runs(X, Y):
    gammas = [0.01, 0.1, 1]
    etas = [0.01, 0.1, 1, 10, 100]
    all_diffs, all_obj = [], []
    for j in range(len(gammas)):
        gamma_diffs = []
        objs = []
        for i in range(len(etas)):
            if j == 0:
                diffs, objective = mgd(gammas[j], etas[i], X, Y)
                gamma_diffs.append(diffs), objs.append(objective)
            elif j ==1 and i != 4:
                diffs, objective = mgd(gammas[j], etas[i], X, Y)
                gamma_diffs.append(diffs), objs.append(objective)
            elif j ==2 and i < 3:
                diffs, objective = mgd(gammas[j], etas[i], X, Y)
                gamma_diffs.append(diffs), objs.append(objective)
            else:
                pass
        all_obj.append(objs)
        #print(j, i, len(gamma_diffs), len(gamma_diffs[0]))
        all_diffs.append(gamma_diffs)
    return all_diffs, all_obj

def graph_em(all_diffs, all_obj):
    gammas = [0.01, 0.1, 1]
    etas = [0.01, 0.1, 1, 10, 100]
    time = [i for i in range(1000)]
    print(all_obj)
    for i in range(len(all_diffs)):
        for j in range(len(all_diffs[i])):
            plt.plot(time, all_diffs[i][j], label="eta="+str(etas[j]))
            print("gamma", gammas[i], "eta", etas[j], "objective", all_obj[i][j])
        plt.title("gamma=" + str(gammas[i]) ) 
        plt.xlabel("t")
        plt.ylabel("log distance between f(w_t) and f_star")                    
        plt.legend()
        plt.savefig("log_dist_for_gamma_"+ str(gammas[i])+".png")
        plt.close()
    
X = samples[:, :-1]
Y = np.reshape(samples[:, -1], (len(samples[:, -1]), 1))
all_diffs, all_obj = collect_runs(X, Y)
graph_em(all_diffs, all_obj)

[[0.2505453990840165, 0.08201220206247475, 0.020031235087500012, 0.0013178370047732036, 3.43049483319885e-06], [0.2540606598125504, 0.09165519373544956, 0.029039781597780936, 0.006055224879481706], [0.25440612039766985, 0.09242721052738719, 0.029683627704031927]]
gamma 0.01 eta 0.01 objective 0.2505453990840165
gamma 0.01 eta 0.1 objective 0.08201220206247475
gamma 0.01 eta 1 objective 0.020031235087500012
gamma 0.01 eta 10 objective 0.0013178370047732036
gamma 0.01 eta 100 objective 3.43049483319885e-06
gamma 0.1 eta 0.01 objective 0.2540606598125504
gamma 0.1 eta 0.1 objective 0.09165519373544956
gamma 0.1 eta 1 objective 0.029039781597780936
gamma 0.1 eta 10 objective 0.006055224879481706
gamma 1 eta 0.01 objective 0.25440612039766985
gamma 1 eta 0.1 objective 0.09242721052738719
gamma 1 eta 1 objective 0.029683627704031927


In [4]:
import matplotlib.pyplot as plt

def stochastic_update(learning_rate, batch_size, w, data):
    #     print((learning_rate*g_t(gamma, old_g, w, y, x)).shape, "update")
    # need to sample a batch_size of points from X
    sample = data[np.random.randint(data.shape[0], size=batch_size), :]
    X = sample[:, :-1]
    Y = np.reshape(sample[:, -1], (len(sample[:, -1]), 1))

    full_gradient = []
    for i in range(X.shape[0]):
        gradient = (1/(1+np.exp((Y[i] * (X[i] @ w))))) * (-X[i] * Y[i])
        full_gradient.append(gradient)
    gradient = np.sum(full_gradient, axis=0)* (1/X.shape[0])
    gradient = np.reshape(gradient, (len(gradient), 1))
    w = w - learning_rate*gradient
    assert w.shape == (100, 1), "w's shape has been changed"
    assert X.shape == (batch_size, 100), "x's shape has been changed"
    assert Y.shape == (batch_size, 1), "y's shape has been changed"
    return w

                  
def objective(data, w):
    X = data[:, :-1]
    Y = np.reshape(data[:, -1], (len(data[:, -1]), 1))
    np.seterr(over='ignore')
    #print(X.shape, Y.shape, w.shape)
    obj = (1/X.shape[0])*np.sum(np.log(1+np.exp(np.multiply(-Y, X@w))))
    return obj

# what am I supposed to do with the vals
def sgd(batch_size, learning_rate, data):    
    
    diffs = []
    for epoch in range(25):
        epoch_diffs = []
        w = np.zeros((X.shape[1], 1))
        for i in range(500):
            w = stochastic_update(learning_rate, batch_size, w, data)
            epoch_diffs.append(calc_diff(w, data))
        diffs.append(epoch_diffs)
    last_obj = objective(data, w)
    return diffs, last_obj


def calc_diff(w, data):
    return np.log(objective(data, w) - 2.906898597653035e-06)

def collect_runs(data):
    batch_sizes = [1,10,100,1000]
    etas = [1, 0.3, 0.1, 0.03]
    all_avg_diffs = []
    for j in range(len(batch_sizes)):
        batch_diffs = []
        for i in range(len(etas)):
            diffs, last_obj = sgd(batch_sizes[j], etas[i], data)
            print("batch_size", batch_sizes[j], "eta", etas[i], "objective", last_obj)
            diffs = np.array(diffs)
            # get the avg
            #print(diffs.shape)
            all_var = np.mean(diffs, axis=0)
            #print(all_var.shape)
            batch_diffs.append(all_var)
        all_avg_diffs.append(batch_diffs)
    return all_avg_diffs

def graph_em(all_avg_objs):
    batch_sizes = [1,10,100,1000]
    etas = [1, 0.3, 0.1, 0.03]
    time = [i for i in range(500)]
    for i in range(len(all_avg_objs)):
        for j in range(len(all_avg_objs[i])):
            # this will need more advanced logic
            plt.plot(time, all_avg_objs[i][j], label="eta="+str(etas[j]))
        plt.title("batch_size=" + str(batch_sizes[i])) 
        plt.xlabel("t")
        plt.ylabel("log distance between f(w_t) and f_star")                    
        plt.legend()
        plt.savefig("log_dist_for_batch_size_"+ str(batch_sizes[i])+".png")
        plt.close()
    # 
    for j in range(len(all_avg_objs[0])):
        for i in range(len(all_avg_objs)):
            # this will need more advanced logic
            plt.plot(time, all_avg_objs[i][j], label="batch_size=" + str(batch_sizes[i]))
        plt.title("eta="+str(etas[j])) 
        plt.xlabel("t")
        plt.ylabel("log distance between f(w_t) and f_star")                    
        plt.legend()
        plt.savefig("log_dist_for_learning_rate_"+ str(etas[j])+".png")
        plt.close()
    # will need a second set of graphs
                   
    
all_obj= collect_runs(samples)
graph_em(all_obj)

batch_size 1 eta 1 objective 4.496179268165029
batch_size 1 eta 0.3 objective 0.9877771065037633
batch_size 1 eta 0.1 objective 0.35739901560532156
batch_size 1 eta 0.03 objective 0.25054809882619455
batch_size 10 eta 1 objective 0.08956013241793022
batch_size 10 eta 0.3 objective 0.0875961519804719
batch_size 10 eta 0.1 objective 0.13308225622946246
batch_size 10 eta 0.03 objective 0.2188798004880944
batch_size 100 eta 1 objective 0.044257433829919396
batch_size 100 eta 0.3 objective 0.07702514516162857
batch_size 100 eta 0.1 objective 0.12727941895043532
batch_size 100 eta 0.03 objective 0.21496493052549362
batch_size 1000 eta 1 objective 0.042813358211889724
batch_size 1000 eta 0.3 objective 0.07657261420817076
batch_size 1000 eta 0.1 objective 0.12673429127223354
batch_size 1000 eta 0.03 objective 0.21450404473198176
